# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at mlta-2022-spring@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/1H5ZONrb2LMOCixLY7D5_5-7LkIaXO6AGEaV2mRdTOMY/edit?usp=sharing)　Kaggle: [Link](https://www.kaggle.com/c/ml2022spring-hw7)　Data: [Link](https://drive.google.com/uc?id=1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb)




## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2.5hrs
  

## Download Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# Download link 1
!gdown --id '1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb' --output hw7_data.zip

# Download Link 2 (if the above link fails) 
# !gdown --id '1qwjbRjq481lHsnTrrF4OjKQnxzgoLEFR' --output hw7_data.zip

# Download Link 3 (if the above link fails) 
# !gdown --id '1QXuWjNRZH6DscSd6QcRER0cnxmpZvijn' --output hw7_data.zip

!unzip -o hw7_data.zip

# For this HW, K80 < P4 < T4 < P100 <= T4(fp16) < V100
!nvidia-smi

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1AVgZvy3VFeg0fX-6WQJMHPVrx3A-M1kb
To: /content/hw7_data.zip
100% 9.57M/9.57M [00:00<00:00, 261MB/s]
Archive:  hw7_data.zip
  inflating: hw7_dev.json            
  inflating: hw7_test.json           
  inflating: hw7_train.json          
Thu May  5 11:48:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      

## Install transformers

Documentation for the toolkit:　https://huggingface.co/transformers/

In [3]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers==4.5.0

     |████████████████████████████████| 2.1 MB 8.6 MB/s 
     |████████████████████████████████| 880 kB 56.0 MB/s 
     |████████████████████████████████| 3.3 MB 52.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=4176279b717258f39654736bd54d733253e9472854856c191d56f62590ee03fd
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


## Import Packages

In [4]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	  torch.manual_seed(seed)
	  if torch.cuda.is_available():
		    torch.cuda.manual_seed(seed)
		    torch.cuda.manual_seed_all(seed)
	  np.random.seed(seed)
	  random.seed(seed)
	  torch.backends.cudnn.benchmark = False
	  torch.backends.cudnn.deterministic = True
same_seeds(0)

In [5]:
# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = True

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

     |████████████████████████████████| 47 kB 3.0 MB/s 


## Load Model and Tokenizer




 

In [6]:
# model_save_dir = "/content/drive/MyDrive/saved_model/roberta" 
# model = BertForQuestionAnswering.from_pretrained(model_save_dir).to(device)

# model = BertForQuestionAnswering.from_pretrained("bert-base-chinese").to(device)
# tokenizer = BertTokenizerFast.from_pretrained("bert-base-chinese")

# model = BertForQuestionAnswering.from_pretrained("hfl/chinese-macbert-large").to(device)
# tokenizer = BertTokenizerFast.from_pretrained("hfl/chinese-macbert-large")

# model = BertForQuestionAnswering.from_pretrained("hfl/chinese-roberta-wwm-ext-large").to(device)
# tokenizer = BertTokenizerFast.from_pretrained("hfl/chinese-roberta-wwm-ext-large")

# model = BertForQuestionAnswering.from_pretrained("luhua/chinese_pretrain_mrc_roberta_wwm_ext_large").to(device)
tokenizer = BertTokenizerFast.from_pretrained("luhua/chinese_pretrain_mrc_roberta_wwm_ext_large")

# model = BertForQuestionAnswering.from_pretrained("luhua/chinese_pretrain_mrc_macbert_large").to(device)
# tokenizer = BertTokenizerFast.from_pretrained("luhua/chinese_pretrain_mrc_macbert_large")

# model = BertForQuestionAnswering.from_pretrained("uer/roberta-base-chinese-extractive-qa").to(device)
# tokenizer = BertTokenizerFast.from_pretrained("uer/roberta-base-chinese-extractive-qa")

# model = BertForQuestionAnswering.from_pretrained("deepset/roberta-large-squad2").to(device)
# tokenizer = BertTokenizerFast.from_pretrained("deepset/roberta-large-squad2")


# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/269k [00:00<?, ?B/s]

## Read Data

- Training set: 31690 QA pairs
- Dev set: 4131  QA pairs
- Test set: 4957  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [10]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

## Tokenize Data

In [11]:
test_paragraphs = [i.replace(' ','％').replace('\u200b','＆').replace('\u200e', '＝').replace('\u3000', '＊') for i in test_paragraphs]
dev_paragraphs = [i.replace(' ','％').replace('\u200b','＆').replace('\u200e', '＝').replace('\u3000', '＊') for i in dev_paragraphs]


# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

## Dataset and Dataloader

In [12]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 50
        self.max_paragraph_len = 300
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = 150

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: Preprocessing #####
        # Hint: How to prevent model from learning something it should not learn

        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

            # A single window is obtained by slicing the portion of paragraph containing the answer
            mid = (answer_start_token + answer_end_token) // 2
            # paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            min_paragraph_start = max(0, answer_end_token - self.max_paragraph_len)
            max_paragraph_start = min(answer_start_token, max(0, len(tokenized_paragraph) - self.max_paragraph_len))
            paragraph_start = random.randint(min_paragraph_start, max_paragraph_start)

            period = []
            for i in range(max(0, min_paragraph_start-1), max_paragraph_start+1):
                if i == 0:
                    period.append(0)
                if tokenizer.decode(tokenized_paragraph.ids[i]) == '。' or tokenizer.decode(tokenized_paragraph.ids[i]) == '，' or tokenizer.decode(tokenized_paragraph.ids[i]) == '？' or tokenizer.decode(tokenized_paragraph.ids[i]) == '！':
                    period.append(i + 1)
                if tokenizer.decode(tokenized_paragraph.ids[i]) == '「':
                    period.append(i)
            if period:
                paragraph_start = period[random.randint(0, len(period) - 1)]

            paragraph_end = paragraph_start + self.max_paragraph_len
            
            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

train_batch_size = 10

# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

In [13]:
dev_set[84][1].shape

torch.Size([3, 353])

In [14]:
import re
def brute_force(par, ans):
    ans = ans.replace('[UNK]', '(.)+')
    if ans[-1] == '+':
        ans = ans[:-1]
    s = re.search(ans, par)
    if s :
        ans = s.group(0)
    return (ans)

def token_index(tokens, ans):
    ans = ans.split(' ')
    start = 0
    end = 0
    for i in range(len(tokens)):
        if tokens[i].replace('#','') == ans[0]:
            k=0
            while k < len(ans) and i+k < len(tokens) and tokens[i+k].replace('#','') == ans[k]:
                k+=1
            if k==len(ans):
                return i, i + k -1
    return 0,0
    
def new_index(tokens, start, end):
    cnt = 0
    new_start = 0
    new_end = 0
    check = 0
        
    for i in range(len(tokens)):
        if tokens[i] == '[UNK]' or tokens[i] == '[CLS]' or tokens[i] == '[SEP]':
            if i == start:
                new_start = cnt
            if i == end:
                new_end = cnt
            cnt += 1
        else:
            for c in tokens[i]:
                if i == start and check == 0:
                    new_start = cnt
                    check = 1
                if i == end:
                    new_end = cnt
                if c != '#':
                    cnt += 1
    return new_start, new_end

Reference : https://github.com/pai4451/ML2021  
Reference : https://github.com/liver121888/NTUEE-2021-Machine-Learning-Assignments/tree/master/HW7

## Function for Evaluation

In [ ]:
def evaluate(data, output, paragraph=None, paragraph_tokenized=None):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    # print('num of win ', num_of_windows)

    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)

        pre_max_val = 0
        pre_max = -10
        pre_max_list = []
        pre_max_vallist = []
        for i in range(len(output.start_logits[k])):
            if output.start_logits[k][i] >= pre_max_val and tokenizer.decode(data[0][0][k][i]) != '[CLS]' and tokenizer.decode(data[0][0][k][i]) != '[SEP]':
                pre_max = i
                pre_max_val = output.start_logits[k][i]
            pre_max_list.append(pre_max)
            pre_max_vallist.append(pre_max_val)
        
        prob = pre_max_vallist[end_index] + end_prob
        if prob > max_prob and tokenizer.decode(data[0][0][k][pre_max_list[end_index]]) != '[CLS]' and tokenizer.decode(data[0][0][k][pre_max_list[end_index]]) != '[SEP]' and tokenizer.decode(data[0][0][k][end_index]) != '[SEP]':
            max_prob = prob
            answer = tokenizer.decode(data[0][0][k][pre_max_list[end_index] : end_index + 1])
            if '[CLS]' in answer:
                print(tokenizer.decode(data[0][0][k][pre_max_list[end_index]]))

        post_max_val = 0
        post_max = -10
        post_max_list = []
        post_max_vallist = []
        for i in range(len(output.end_logits[k]) - 1, -1, -1):
            if output.end_logits[k][i] >= post_max_val and tokenizer.decode(data[0][0][k][i]) != '[CLS]' and tokenizer.decode(data[0][0][k][i]) != '[SEP]' :
                post_max = i
                post_max_val = output.end_logits[k][i]
            post_max_list.append(post_max)
            post_max_vallist.append(post_max_val)
        
        post_max_list.reverse()
        post_max_vallist.reverse()

        prob = post_max_vallist[start_index] + start_prob
        if prob > max_prob and tokenizer.decode(data[0][0][k][start_index]) != '[CLS]' and tokenizer.decode(data[0][0][k][start_index]) != '[SEP]' and tokenizer.decode(data[0][0][k][post_max_list[start_index]]) != '[SEP]':
            max_prob = prob
            answer = tokenizer.decode(data[0][0][k][start_index : post_max_list[start_index]+1])
            if '[CLS]' in answer:
                print(tokenizer.decode(data[0][0][k][start_index]))

    if '[UNK]' in answer:
        print(answer)
        tmp_start_index, tmp_end_index = token_index(tokens=paragraph_tokenized, ans=answer)
        new_start, new_end = new_index(tokens=paragraph_tokenized, start=tmp_start_index, end=tmp_end_index)
        if paragraph[new_start] == '，':
            new_start += 1
            new_end += 1
        if not tmp_start_index and not tmp_end_index:
            answer = answer.replace(' ','')
            answer = brute_force(paragraph, answer)
        else:
            answer = paragraph[new_start : new_end+1]
        
        print(answer)
    answer = answer.replace('[PAD]','')
    answer = answer.replace('％', ' ').replace('＆','\u200b').replace('＝','\u200e').replace('＊','\u3000')


    return answer.replace(' ','')

## Training

In [ ]:
import transformers
num_epoch = 3
validation = False
logging_step = 100
learning_rate = 2e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)

# sch=transformers.get_linear_schedule_with_warmup(optimizer, 10, 3 * len(train_loader))


if fp16_training:
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 

if not validation:
    dev_set = QA_Dataset("train", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
    train_set = torch.utils.data.ConcatDataset([train_set, dev_set])
    train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
total_stp = num_epoch * len(train_loader)

model.train()
accu = 3
print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0
    for data in tqdm(train_loader):	
        
        # Load all data into GPU
        data = [i.to(device) for i in data]
        
        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])

        # Choose the most probable start position / end position
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)
        
        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
        train_loss += output.loss
        
        

        if fp16_training:
            # output.loss = output.loss / accu
            accelerator.backward(output.loss)
        else:
            # output.loss = output.loss / accu
            output.loss.backward()

        # if (step % accu == 0) or (step == len(train_loader)):

        optimizer.step()
        optimizer.param_groups[0]['lr'] -= (learning_rate / total_stp)
        # sch.step()
        optimizer.zero_grad()
        step += 1
        ##### TODO: Apply linear learning rate decay #####
        
        
        # Print training loss and accuracy over past logging step
        if step % logging_step == 0:
            print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            train_loss = train_acc = 0
            print(optimizer.param_groups[0]['lr'])

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output,  dev_paragraphs[dev_questions[i]['paragraph_id']], dev_paragraphs_tokenized[dev_questions[i]['paragraph_id']].tokens) == dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()

# Save a model and its configuration file to the directory 「saved_model」 
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
print("Saving Model ...")
model_save_dir = "/content/drive/MyDrive/saved_model/roberta_period2"
model.save_pretrained(model_save_dir)

Start Training ...


  0%|          | 0/3583 [00:00<?, ?it/s]

Epoch 1 | Step 100 | loss = 0.734, acc = 0.714
1.9815796818308573e-05
Epoch 1 | Step 200 | loss = 0.623, acc = 0.738
1.962973299841824e-05
Epoch 1 | Step 300 | loss = 0.613, acc = 0.739
1.9443669178527908e-05
Epoch 1 | Step 400 | loss = 0.601, acc = 0.770
1.9257605358637575e-05
Epoch 1 | Step 500 | loss = 0.607, acc = 0.753
1.9071541538747243e-05
Epoch 1 | Step 600 | loss = 0.656, acc = 0.741
1.888547771885691e-05
Epoch 1 | Step 700 | loss = 0.592, acc = 0.756
1.869941389896658e-05
Epoch 1 | Step 800 | loss = 0.576, acc = 0.765
1.8513350079076246e-05
Epoch 1 | Step 900 | loss = 0.534, acc = 0.786
1.8327286259185914e-05
Epoch 1 | Step 1000 | loss = 0.526, acc = 0.784
1.814122243929558e-05
Epoch 1 | Step 1100 | loss = 0.515, acc = 0.773
1.795515861940525e-05
Epoch 1 | Step 1200 | loss = 0.490, acc = 0.791
1.7769094799514917e-05
Epoch 1 | Step 1300 | loss = 0.546, acc = 0.781
1.7583030979624584e-05
Epoch 1 | Step 1400 | loss = 0.549, acc = 0.785
1.7396967159734252e-05
Epoch 1 | Step 1500 

  0%|          | 0/3583 [00:00<?, ?it/s]

Epoch 2 | Step 100 | loss = 0.226, acc = 0.875
1.3149130151639885e-05
Epoch 2 | Step 200 | loss = 0.226, acc = 0.898
1.2963066331749722e-05
Epoch 2 | Step 300 | loss = 0.206, acc = 0.897
1.2777002511859559e-05
Epoch 2 | Step 400 | loss = 0.206, acc = 0.902
1.2590938691969396e-05
Epoch 2 | Step 500 | loss = 0.227, acc = 0.908
1.2404874872079233e-05
Epoch 2 | Step 600 | loss = 0.186, acc = 0.910
1.221881105218907e-05
Epoch 2 | Step 700 | loss = 0.210, acc = 0.905
1.2032747232298907e-05
Epoch 2 | Step 800 | loss = 0.223, acc = 0.890
1.1846683412408744e-05
Epoch 2 | Step 900 | loss = 0.241, acc = 0.901
1.1660619592518581e-05
Epoch 2 | Step 1000 | loss = 0.174, acc = 0.907
1.1474555772628418e-05
Epoch 2 | Step 1100 | loss = 0.186, acc = 0.898
1.1288491952738255e-05
Epoch 2 | Step 1200 | loss = 0.231, acc = 0.894
1.1102428132848092e-05
Epoch 2 | Step 1300 | loss = 0.190, acc = 0.906
1.091636431295793e-05
Epoch 2 | Step 1400 | loss = 0.182, acc = 0.903
1.0730300493067766e-05
Epoch 2 | Step 15

  0%|          | 0/3583 [00:00<?, ?it/s]

Epoch 3 | Step 100 | loss = 0.063, acc = 0.959
6.482463484974823e-06
Epoch 3 | Step 200 | loss = 0.089, acc = 0.955
6.296399665084576e-06
Epoch 3 | Step 300 | loss = 0.087, acc = 0.964
6.110335845194328e-06
Epoch 3 | Step 400 | loss = 0.072, acc = 0.969
5.92427202530408e-06
Epoch 3 | Step 500 | loss = 0.090, acc = 0.955
5.738208205413833e-06
Epoch 3 | Step 600 | loss = 0.079, acc = 0.960
5.552144385523585e-06
Epoch 3 | Step 700 | loss = 0.075, acc = 0.968
5.366080565633337e-06
Epoch 3 | Step 800 | loss = 0.076, acc = 0.961
5.18001674574309e-06
Epoch 3 | Step 900 | loss = 0.080, acc = 0.968
4.993952925852842e-06
Epoch 3 | Step 1000 | loss = 0.077, acc = 0.955
4.807889105962594e-06
Epoch 3 | Step 1100 | loss = 0.077, acc = 0.964
4.621825286072347e-06
Epoch 3 | Step 1200 | loss = 0.086, acc = 0.965
4.435761466182099e-06
Epoch 3 | Step 1300 | loss = 0.077, acc = 0.966
4.2496976462918514e-06
Epoch 3 | Step 1400 | loss = 0.079, acc = 0.955
4.063633826401604e-06
Epoch 3 | Step 1500 | loss = 0

## Testing

In [ ]:
print("Evaluating Test Set ...")
# model = BertForQuestionAnswering.from_pretrained('/content/drive/MyDrive/saved_model/macbert').to(device)

result = []
model.eval()
with torch.no_grad():
    for i, data in enumerate(tqdm(test_loader)):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output, test_paragraphs[test_questions[i]['paragraph_id']],
                               test_paragraphs_tokenized[test_questions[i]['paragraph_id']].tokens))

result_file = "/content/drive/MyDrive/result_bert.csv"
with open(result_file, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
		    f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


  0%|          | 0/4957 [00:00<?, ?it/s]

溥 [UNK]
溥儁
貴 州 省 最 大 的 體 育 場 所 [UNK] 「 貴 陽 市 體 育 場 」
貴州省最大的體育場所—「貴陽市體育場」
目 前 沒 有 觀 察 到 任 何 語 言 純 [UNK] 以 力 道 來 區 分 不 同 輔 音
目前沒有觀察到任何語言純綷以力道來區分不同輔音
[UNK] 人 國
荇人國
欸 樓 逼 [UNK] 1806 - 20
欸樓逼咇1806-20
馬 [UNK]
馬馼
常 [UNK]
常璩
[UNK] 稻
秈稻
白 [UNK] 紀 滅 絕 事 件
白堊紀滅絕事件
抗 佝 [UNK] 病 維 他 命
抗佝僂病維他命
杭 州 [UNK] 橋 機 場
杭州筧橋機場
蔡 [UNK]
蔡鍔
丁 [UNK]
丁旿
隋 [UNK] 帝
隋煬帝
胡 季 [UNK]
胡季犛
其 英 文 縮 寫 首 字 母 為 「 [UNK] · ㄎㄟ · ㄨㄞ 」
其英文縮寫首字母為「ㄟㄙ·ㄎㄟ·ㄨㄞ」
梁 [UNK]
梁鵠
[UNK] 靼 海 峽
韃靼海峽
白 [UNK] 紀 末 滅 絕 事 件
白堊紀末滅絕事件
侏 [UNK] 紀
侏儸紀
具 高 商 業 價 值 的 鮭 魚 總 共 有 9 種 ， 來 自 兩 個 不 同 的 屬 。 在 中 國 大 陸 南 方 主 要 是 進 口 的 大 西 洋 鮭 為 主 ， 而 北 方 太 平 洋 品 種 的 大 馬 哈 魚 常 於 9 － 10 月 回 到 黑 龍 江 、 烏 蘇 里 江 等 河 中 產 卵 ， 故 又 叫 「 秋 鮭 」 。 部 分 商 人 會 利 用 品 種 差 異 以 虹 [UNK] 混 充 鮭 魚 。 [UNK] 屬 和 太 平 洋 鮭 屬 都 同 時 有 不 少 魚 種 被 稱 作 [UNK] 魚 。 此 外 ， [UNK] 屬 的 鈍 吻 鮭 和 隆 頭 [UNK] 在 英 語 國 家 中 都 會 視 爲 鮭 魚 的 其 中 一 種 。 雖 然 虹 [UNK] 會 從 淡 水 移 居 至 海 水 ， 不 過 一 般 不 視 為 鮭 魚 的 一 種 。 此 外 ， 還 有 一 些 魚 類 也 會 被 稱 作 鮭 魚 。 除 了 多 瑙 哲 羅 魚 為 鮭 科 大 型 淡 水 魚 外 ， 其 他 魚 種 都 屬 於 鱸 形 目 。 [SEP] [PAD] [PAD] [PAD] [PAD] [

ensemble

In [15]:
def evaluate_ensemble(data, output1, output2, output3, output4, paragraph=None, paragraph_tokenized=None):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    # print('num of win ', num_of_windows)
    start_logits = (output1.start_logits + output2.start_logits + output3.start_logits + output4.start_logits)
    end_logits = (output1.end_logits + output2.end_logits + output3.end_logits + output4.end_logits)
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        
        start_prob, start_index = torch.max(start_logits[k], dim=0)
        end_prob, end_index = torch.max(end_logits[k], dim=0)

        ##old approach
        # pre_max_val = 0
        # pre_max = -10
        # pre_max_list = []
        # pre_max_vallist = []
        # for i in range(len(start_logits[k])):
        #     if start_logits[k][i] >= pre_max_val and tokenizer.decode(data[0][0][k][i]) != '[CLS]' and tokenizer.decode(data[0][0][k][i]) != '[SEP]':
        #         pre_max = i
        #         pre_max_val = start_logits[k][i]
        #     pre_max_list.append(pre_max)
        #     pre_max_vallist.append(pre_max_val)
        
        # prob = pre_max_vallist[end_index] + end_prob
        # if prob > max_prob and tokenizer.decode(data[0][0][k][pre_max_list[end_index]]) != '[CLS]' and tokenizer.decode(data[0][0][k][pre_max_list[end_index]]) != '[SEP]' and tokenizer.decode(data[0][0][k][end_index]) != '[SEP]':
        #     max_prob = prob
        #     answer = tokenizer.decode(data[0][0][k][pre_max_list[end_index] : end_index + 1])
        #     if '[CLS]' in answer:
        #         print(tokenizer.decode(data[0][0][k][pre_max_list[end_index]]))

        # post_max_val = 0
        # post_max = -10
        # post_max_list = []
        # post_max_vallist = []
        # for i in range(len(end_logits[k]) - 1, -1, -1):
        #     if end_logits[k][i] >= post_max_val and tokenizer.decode(data[0][0][k][i]) != '[CLS]' and tokenizer.decode(data[0][0][k][i]) != '[SEP]':
        #         post_max = i
        #         post_max_val = end_logits[k][i]
        #     post_max_list.append(post_max)
        #     post_max_vallist.append(post_max_val)
        
        # post_max_list.reverse()
        # post_max_vallist.reverse()

        # prob = post_max_vallist[start_index] + start_prob
        # if prob > max_prob and tokenizer.decode(data[0][0][k][start_index]) != '[CLS]' and tokenizer.decode(data[0][0][k][start_index]) != '[SEP]' and tokenizer.decode(data[0][0][k][post_max_list[start_index]]) != '[SEP]':
        #     max_prob = prob
        #     answer = tokenizer.decode(data[0][0][k][start_index : post_max_list[start_index]+1])
        #     if '[CLS]' in answer:
        #         print(tokenizer.decode(data[0][0][k][start_index]))

        # prefix_max = start_logits[k][0]
        # prefix_max_index = 0
        # prob = 0
        # for i in range(1, len(start_logits[k])):
        #     if end_logits[k][i] + prefix_max >= prob:
        #         start_index = prefix_max_index
        #         end_index = i
        #     if start_logits[k][i] > prefix_max:
        #         # update max start logit
        #         prefix_max = start_logits[k][i]
        #         prefix_max_index = i
        
        prefix_max_index = 0
        prob = 0
        for i in range(1, len(start_logits[k])):
            if end_logits[k][i] + start_logits[k][prefix_max_index] >= prob:
                start_index = prefix_max_index
                end_index = i
                prob = end_logits[k][end_index] + start_logits[k][prefix_max_index]
            if start_logits[k][i] > start_logits[k][prefix_max_index]:
                # update max start logit
                prefix_max_index = i

        if prob > max_prob and tokenizer.decode(data[0][0][k][start_index]) != '[CLS]' and tokenizer.decode(data[0][0][k][start_index]) != '[SEP]' and tokenizer.decode(data[0][0][k][end_index]) != '[SEP]':
            max_prob = prob
            answer = tokenizer.decode(data[0][0][k][start_index : end_index+1])


    if '[UNK]' in answer:
        print(answer)
        tmp_start_index, tmp_end_index = token_index(tokens=paragraph_tokenized, ans=answer)
        new_start, new_end = new_index(tokens=paragraph_tokenized, start=tmp_start_index, end=tmp_end_index)
        if paragraph[new_start] == '，':
            new_start += 1
            new_end += 1
        if tmp_start_index == tmp_end_index:
            answer = answer.replace(' ','')
            answer = brute_force(paragraph, answer)
        else:
            answer = paragraph[new_start : new_end+1]
        
        print(answer)
    answer = answer.replace('[PAD]','')
    answer = answer.replace('％', ' ').replace('＆','\u200b').replace('＝','\u200e').replace('＊','\u3000')

    return answer.replace(' ','')

In [16]:
print("Evaluating Test Set ...")
model1 = BertForQuestionAnswering.from_pretrained('/content/drive/MyDrive/saved_model/macbert_period').to(device)
model2 = BertForQuestionAnswering.from_pretrained('/content/drive/MyDrive/saved_model/macbert_period2').to(device)
model3 = BertForQuestionAnswering.from_pretrained('/content/drive/MyDrive/saved_model/roberta_period').to(device)
model4 = BertForQuestionAnswering.from_pretrained('/content/drive/MyDrive/saved_model/roberta_period2').to(device)
# model5 = BertForQuestionAnswering.from_pretrained('/content/drive/MyDrive/saved_model/roberta3').to(device)

result = []
model1.eval()
model2.eval()
model3.eval()
model4.eval()
# model5.eval()
with torch.no_grad():
    for i, data in enumerate(tqdm(test_loader)):
        output1 = model1(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        output2 = model2(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        output3 = model3(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        output4 = model4(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        # output5 = model5(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
        #                attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate_ensemble(data, output1, output2, output3, output4, test_paragraphs[test_questions[i]['paragraph_id']],
                               test_paragraphs_tokenized[test_questions[i]['paragraph_id']].tokens))

result_file = "/content/drive/MyDrive/result_bert.csv"
with open(result_file, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
		    f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


  0%|          | 0/4957 [00:00<?, ?it/s]

溥 [UNK]
溥儁
目 前 沒 有 觀 察 到 任 何 語 言 純 [UNK] 以 力 道 來 區 分 不 同 輔 音
目前沒有觀察到任何語言純綷以力道來區分不同輔音
[UNK] 人 國
荇人國
欸 樓 逼 [UNK] 1806 - 20
欸樓逼咇1806-20
馬 [UNK]
馬馼
東 晉 常 [UNK]
東晉常璩
[UNK] 稻
秈稻
白 [UNK] 紀 滅 絕 事 件
白堊紀滅絕事件
抗 佝 [UNK] 病 維 他 命
抗佝僂病維他命
杭 州 [UNK] 橋 機 場
杭州筧橋機場
蔡 [UNK]
蔡鍔
丁 [UNK]
丁旿
隋 [UNK] 帝
隋煬帝
胡 季 [UNK]
胡季犛
其 英 文 縮 寫 首 字 母 為 「 [UNK] · ㄎㄟ · ㄨㄞ 」
其英文縮寫首字母為「ㄟㄙ·ㄎㄟ·ㄨㄞ」
梁 [UNK]
梁鵠
[UNK] 靼 海 峽
韃靼海峽
白 [UNK] 紀 末 滅 絕 事 件
白堊紀末滅絕事件
侏 [UNK] 紀
侏儸紀
克 里 米 亞 [UNK] 靼 人
克里米亞韃靼人
白 [UNK] 紀 中 期
白堊紀中期
白 [UNK] 紀
白堊紀
白 [UNK] 紀
白堊紀
Completed! Result is in /content/drive/MyDrive/result_bert.csv


In [ ]:
def evaluate_ensemble(data, output1, output2, paragraph=None, paragraph_tokenized=None):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    # print('num of win ', num_of_windows)
    start_logits = (output1.start_logits + output2.start_logits)
    end_logits = (output1.end_logits + output2.end_logits)
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        
        start_prob, start_index = torch.max(start_logits[k], dim=0)
        end_prob, end_index = torch.max(end_logits[k], dim=0)


        pre_max_val = 0
        pre_max = -10
        pre_max_list = []
        pre_max_vallist = []
        for i in range(len(start_logits[k])):
            if start_logits[k][i] >= pre_max_val and tokenizer.decode(data[0][0][k][i]) != '[CLS]' and tokenizer.decode(data[0][0][k][i]) != '[SEP]':
                pre_max = i
                pre_max_val = start_logits[k][i]
            pre_max_list.append(pre_max)
            pre_max_vallist.append(pre_max_val)
        
        prob = pre_max_vallist[end_index] + end_prob
        if prob > max_prob and tokenizer.decode(data[0][0][k][pre_max_list[end_index]]) != '[CLS]' and tokenizer.decode(data[0][0][k][pre_max_list[end_index]]) != '[SEP]' and tokenizer.decode(data[0][0][k][end_index]) != '[SEP]':
            max_prob = prob
            answer = tokenizer.decode(data[0][0][k][pre_max_list[end_index] : end_index + 1])
            if '[CLS]' in answer:
                print(tokenizer.decode(data[0][0][k][pre_max_list[end_index]]))

        post_max_val = 0
        post_max = -10
        post_max_list = []
        post_max_vallist = []
        for i in range(len(end_logits[k]) - 1, -1, -1):
            if end_logits[k][i] >= post_max_val and tokenizer.decode(data[0][0][k][i]) != '[CLS]' and tokenizer.decode(data[0][0][k][i]) != '[SEP]':
                post_max = i
                post_max_val = end_logits[k][i]
            post_max_list.append(post_max)
            post_max_vallist.append(post_max_val)
        
        post_max_list.reverse()
        post_max_vallist.reverse()

        prob = post_max_vallist[start_index] + start_prob
        if prob > max_prob and tokenizer.decode(data[0][0][k][start_index]) != '[CLS]' and tokenizer.decode(data[0][0][k][start_index]) != '[SEP]' and tokenizer.decode(data[0][0][k][post_max_list[start_index]]) != '[SEP]':
            max_prob = prob
            answer = tokenizer.decode(data[0][0][k][start_index : post_max_list[start_index]+1])
            if '[CLS]' in answer:
                print(tokenizer.decode(data[0][0][k][start_index]))

    if '[UNK]' in answer:
        print(answer)
        tmp_start_index, tmp_end_index = token_index(tokens=paragraph_tokenized, ans=answer)
        new_start, new_end = new_index(tokens=paragraph_tokenized, start=tmp_start_index, end=tmp_end_index)
        if paragraph[new_start] == '，':
            new_start += 1
            new_end += 1
        if tmp_start_index == tmp_end_index:
            answer = answer.replace(' ','')
            answer = brute_force(paragraph, answer)
        else:
            answer = paragraph[new_start : new_end+1]
        
        print(answer)
    answer = answer.replace('[PAD]','')
    answer = answer.replace('％', ' ').replace('＆','\u200b').replace('＝','\u200e').replace('＊','\u3000')

    return answer.replace(' ','')

In [ ]:
print("Evaluating Test Set ...")
model1 = BertForQuestionAnswering.from_pretrained('/content/drive/MyDrive/saved_model/macbert_period').to(device)
model2 = BertForQuestionAnswering.from_pretrained('/content/drive/MyDrive/saved_model/macbert_period2').to(device)
# model3 = BertForQuestionAnswering.from_pretrained('/content/drive/MyDrive/saved_model/roberta_period').to(device)
# model4 = BertForQuestionAnswering.from_pretrained('/content/drive/MyDrive/saved_model/roberta_period2').to(device)
# model5 = BertForQuestionAnswering.from_pretrained('/content/drive/MyDrive/saved_model/roberta3').to(device)

result = []
model1.eval()
model2.eval()
# model3.eval()
# model4.eval()
# model5.eval()
with torch.no_grad():
    for i, data in enumerate(tqdm(test_loader)):
        output1 = model1(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        output2 = model2(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        # output3 = model3(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
        #                attention_mask=data[2].squeeze(dim=0).to(device))
        # output4 = model4(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
        #                attention_mask=data[2].squeeze(dim=0).to(device))
        # output5 = model5(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
        #                attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate_ensemble(data, output1, output2, test_paragraphs[test_questions[i]['paragraph_id']],
                               test_paragraphs_tokenized[test_questions[i]['paragraph_id']].tokens))

result_file = "/content/drive/MyDrive/result_bert.csv"
with open(result_file, 'w') as f:	
	  f.write("ID,Answer\n")
	  for i, test_question in enumerate(test_questions):
        # Replace commas in answers with empty strings (since csv is separated by comma)
        # Answers in kaggle are processed in the same way
		    f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


  0%|          | 0/4957 [00:00<?, ?it/s]

溥 [UNK]
溥儁
目 前 沒 有 觀 察 到 任 何 語 言 純 [UNK] 以 力 道 來 區 分 不 同 輔 音
目前沒有觀察到任何語言純綷以力道來區分不同輔音
[UNK]
育
馬 [UNK]
馬馼
東 晉 常 [UNK]
東晉常璩
[UNK] 稻
秈稻
白 [UNK] 紀 滅 絕 事 件
白堊紀滅絕事件
抗 佝 [UNK] 病
抗佝僂病
杭 州 [UNK] 橋 機 場
杭州筧橋機場
蔡 [UNK]
蔡鍔
丁 [UNK]
丁旿
隋 [UNK] 帝
隋煬帝
胡 季 [UNK]
胡季犛
其 英 文 縮 寫 首 字 母 為 「 [UNK] · ㄎㄟ · ㄨㄞ 」
其英文縮寫首字母為「ㄟㄙ·ㄎㄟ·ㄨㄞ」
鯨 魚 座 · 優 [UNK]
鯨魚座·優咇
梁 [UNK]
梁鵠
[UNK] 靼 海 峽
韃靼海峽
白 [UNK] 紀 末 滅 絕 事 件
白堊紀末滅絕事件
侏 [UNK] 紀
侏儸紀
克 里 米 亞 [UNK] 靼 人
克里米亞韃靼人
100 % ， 黏 性 最 高 。 又 分 粳 糯 及 [UNK] 糯 ， 粳 糯 外 觀 圓 短 ， [UNK] 糯 外 觀 細 長 ， 顏 色 均 為 白 色 不 透 明 。 煮 熟 後 米 飯 較 軟 、 黏 。 粳 糯 常 見 用 途 為 用 於 釀 酒 、 米 糕 、 紅 龜 粿 、 甜 粿 、 湯 圓 等 。 [UNK] 糯 則 作 為 八 寶 粥 、 粽 子 、 油 飯 等 原 料 。 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

In [ ]:
re.search('42.00(.)','42.00牀。新北市立聯合醫院爲新北市政府成立的公立醫')